<a href="https://colab.research.google.com/github/HayunJin/open_api/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EC%98%81%ED%99%94%ED%8F%89%EC%A0%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import json
import re
import requests
import pandas as pd

df = pd.DataFrame(columns = [ 'navertitle', 'naversubtitle', 'naveruserScore', 'spScore'])
mv = df
#네이버 검색 Open API 사용 요청시 얻게되는 정보를 입력합니다
naver_client_id = "naver_client_id"
naver_client_secret = "naver_client_secret"
 
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext


#     title = movie
#     director = name + '|'
#     encText = urllib.parse.quote(title)
#     display = '&display=100'
#     yearfrom = '&yearfrom=2010'
#     yearto = '&yearto=2015'
#     url = "https://openapi.naver.com/v1/search/movie?query=" + encText + display + yearfrom + yearto # json 결과
#     # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과

#     url = "https://openapi.naver.com/v1/search/movie?query=" + encText + display + yearfrom + yearto # json 결과

def searchByTitle(title, yearfrom, yearto):
    encText = urllib.parse.quote(title)
    display = '&display=30'
    yearfrom = '&yearfrom='+yearfrom
    yearto = '&yearto='+yearto
    myurl = "https://openapi.naver.com/v1/search/movie?query=" + encText + display + yearfrom + yearto # json 결과
    # myurl = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
    # myurl = 'https://openapi.naver.com/v1/search/movie.json?display=30&query=' + quote('"' + title + '"' + '&yearfrom=' + yearfrom + '&yearto=' + yearto) # 확인필요
    print(myurl)
    request = urllib.request.Request(myurl)
    request.add_header("X-Naver-Client-Id",naver_client_id)
    request.add_header("X-Naver-Client-Secret",naver_client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        d = json.loads(response_body.decode('utf-8'))
        if (len(d['items']) > 0):
            return d['items']
        else:
            return None
 
    else:
        print("Error Code:" + rescode)
 
def findItemByInput(items):
    df = pd.DataFrame(columns = [ 'navertitle', 'naversubtitle', 'naveruserScore', 'spScore'])
    for index, item in enumerate(items):
        navertitle = cleanhtml(item['title'])
        naversubtitle = cleanhtml(item['subtitle'])
        naverpubdate = cleanhtml(item['pubDate'])
        naveractor = cleanhtml(item['actor'])
        naverlink = cleanhtml(item['link'])
        naveruserScore = cleanhtml(item['userRating'])
 
        navertitle1 = navertitle.replace(" ","")
        navertitle1 = navertitle1.replace("-", ",")
        navertitle1 = navertitle1.replace(":", ",")
 
        #기자 평론가 평점을 얻어 옵니다
        spScore = getSpecialScore(naverlink)
 
        #네이버가 다루는 영화 고유 ID를 얻어 옵니다
        naverid = re.split("code=", naverlink)[1]
 
        # 영화의 타이틀 이미지를 표시합니다
        # if (item['image'] != None and "http" in item['image']):
        #    response = requests.get(item['image'])
        #    img = Image.open(BytesIO(response.content))
        #    img.show()
 
        # print(index, navertitle, naversubtitle, naveruserScore, spScore)
        # print(float(naveruserScore), spScore)
        # 
        if (float(naveruserScore) > 0) & (float(spScore) > 0):
            df = df.append({'navertitle' : navertitle
                        , 'naversubtitle' : naversubtitle
                        , 'naveruserScore' : naveruserScore
                        , 'spScore' : spScore} , ignore_index=True)
    return df

 
def getInfoFromNaver(searchTitle, yearfrom, yearto):
    items = searchByTitle(searchTitle, yearfrom, yearto)
 
    if (items != None):
        return findItemByInput(items)
    else:
        print("No result")
 
def get_soup(url):
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'lxml')
    return soup
 
#기자 평론가 평점을 얻어 옵니다
def getSpecialScore(URL):
    soup = get_soup(URL)
    scorearea = soup.find_all('div', "spc_score_area")
    newsoup = BeautifulSoup(str(scorearea), 'lxml')
    score = newsoup.find_all('em')
    if (score and len(score) > 5):
        scoreis = score[1].text + score[2].text + score[3].text + score[4].text
        return float(scoreis)
    else:
        return 0.0
 


In [ ]:
# mv_list = list(set(tmp['movieNm']))
# for i in mv_list:
#     mv = mv.append(getInfoFromNaver(i))


getInfoFromNaver(u'캣츠','2019','2019')

In [ ]:
mv_list = ['해치지않아',
 '신비아파트 극장판 하늘도깨비 대 요르문간드',
 '포드 V 페라리',
 '시동',
 '겨울왕국 2',
 '캣츠',
 '나이브스 아웃',
 '백두산',
 '천문: 하늘에 묻는다',
 '21 브릿지: 테러 셧다운 ',
 '미드웨이']

In [ ]:
# 영화진흥위원회_일별박스오피스
# https://www.kobis.or.kr/kobisopenapi/homepg/apiservice/searchServiceInfo.do

import requests
import json
import pandas as pd
import datetime
from datetime import timedelta

# 0105001
url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key={}&targetDt={}&wideAreaCd=0105001'
# 임시 테이블 생성
tmp = pd.DataFrame(columns = ['audiAcc','audiChange','audiCnt','audiInten','movieCd','movieNm','openDt','rank','rankInten','rankOldAndNew','rnum','salesAcc','salesAmt','salesChange','salesInten','salesShare','scrnCnt','showCnt','targetDt'])

def get_daily_box_office(std_date, i):
    target_dt = (std_date + timedelta(days=i)).strftime("%Y%m%d")
    print(target_dt)
    res = requests.get(url.format("bb8624870e8a6533584586cdcc0ffff0", target_dt))
    text = res.text

    d = json.loads(text) 
    movie = []
    # print(d)

    for b in d['boxOfficeResult']['dailyBoxOfficeList']:
        movie.append([  b['audiAcc'],
                        b['audiChange'],
                        b['audiCnt'],
                        b['audiInten'],
                        b['movieCd'],
                        b['movieNm'],
                        b['openDt'],
                        b['rank'],
                        b['rankInten'],
                        b['rankOldAndNew'],
                        b['rnum'],
                        b['salesAcc'],
                        b['salesAmt'],
                        b['salesChange'],
                        b['salesInten'],
                        b['salesShare'],
                        b['scrnCnt'],
                        b['showCnt']
                        ])
    df = pd.DataFrame(movie,columns = ['audiAcc','audiChange','audiCnt','audiInten','movieCd','movieNm','openDt','rank','rankInten','rankOldAndNew','rnum','salesAcc','salesAmt','salesChange','salesInten','salesShare','scrnCnt','showCnt'])
    df['targetDt'] = target_dt
    return df    

## std_date = datetime.datetime.strptime('조회 시작일', '%Y-%m-%d')
## for i in range(0,조회 하려는 일자 수):
#     tmp = tmp.append(get_daily_box_office(std_date, i))
##
## 예: 조회시작일이 2020-01-01, range(0,2) 인 경우 2020-01-01과 2020-01-02 이틀 조회

std_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d')
for i in range(0,365):
    tmp = tmp.append(get_daily_box_office(std_date, i))


In [ ]:
mv_list = list(set(tmp['movieNm']))

In [ ]:
tmp